In [2]:
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

# 解决中文乱码问题
plt.rcParams["font.sans-serif"]=["SimHei"] 
plt.rcParams["axes.unicode_minus"]=False

In [3]:
data=pd.read_csv('./use_4.csv')
train_data,test_data=train_test_split(data,test_size=0.2, random_state=1318)
feature=train_data.drop(['丙烯选择性','Unnamed: 0'],axis=1)
label=train_data['丙烯选择性']

In [4]:
feature.head()

,Pt,Sn,W,Ni,Hf,V,Ti,Mg,BPO4,B,...,TiO2,Al2O3,CeO2,C2H6,C3H8,O2,保护气体,GHSV,温度,丙烷转化率
14,1.1,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,98.15,0.0,0.0,80.00,4.20,15.80,4760.0,600.0,15.2
70,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,2.44,2.44,95.12,18000.0,540.0,3.0
102,0.0,0.00,0.0,84.4,0.0,0.0,0.0,0.0,0.0,15.6,...,0.0,0.00,0.0,0.0,30.00,15.00,55.00,13645.0,500.0,6.1
213,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,16.70,16.70,66.60,21600.0,450.0,5.0
65,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,30.00,15.00,55.00,436364.0,550.0,9.3


In [5]:
# 预测用数据1
data_predict = feature.copy()
data_predict.iloc[:, :] = 0
data_6 = pd.DataFrame(np.zeros((30000, 33)))
data_6.columns = data_predict.columns
data_predict = pd.concat([data_predict, data_6])
data_predict.iloc[:, 30] = feature.iloc[:, 30].mean()
data_predict.iloc[:, 31] = 510
data_predict.iloc[:, 32] = 5
data_predict.iloc[:, 29] = 10
data_predict.iloc[:, 28] = 20
data_predict.iloc[:, 27] = 70
data_predict.iloc[:, 26] = 0
for i in range(30182):
    x_1 = round(random.uniform(0.1, 10),2)
    data_predict.iloc[i, random.randint(8, 15)] = x_1
    data_predict.iloc[i, random.randint(16, 25)] = 100 - x_1

In [31]:
data_predict.head()

,Mg,Hf,W,V,Ni,Sn,Pt,Ti,CB,P,...,BPO4.1,TiO2,S-1,C2H6,C3H8,O2,保护气体,GHSV,温度,丙烷转化率
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.37,...,98.63,0.0,0.0,0,70,20,10,28197.617804,510,5
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.48,0.00,...,0.00,0.0,0.0,0,70,20,10,28197.617804,510,5
102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,96.13,0.0,0.0,0,70,20,10,28197.617804,510,5
213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.66,0.00,...,94.34,0.0,0.0,0,70,20,10,28197.617804,510,5
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,...,0.00,0.0,0.0,0,70,20,10,28197.617804,510,5


In [32]:
# 结果预测（一元催化剂）
model = cb.CatBoostRegressor(iterations=190,max_depth=8,silent=True,random_state=1314,learning_rate=0.1).fit(X=feature , y=label)
y_hat = pd.DataFrame(model.predict(np.array(data_predict)))
data_predict['y_hat'] = y_hat
data_predict.sort_values(by='y_hat', inplace=True, ascending=False)
data_predict.to_excel('一元催化剂_1.xlsx')